In [ ]:
%matplotlib inline

In [ ]:
from absa.config import DATA_PATHS
from absa.dataset import load_dataset
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.preprocessing import FunctionTransformer, OneHotEncoder
from sklearn.decomposition import TruncatedSVD
from sklearn.model_selection import cross_validate
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline, make_union
from sklearn.svm import SVC
from sklearn.metrics import f1_score
import spacy
nlp = spacy.load('en')

In [ ]:
train_ds_path = DATA_PATHS['asba.semeval16.raw.train']

df_train = load_dataset(train_ds_path)

df_train = df_train.loc[:, ['id', 'text', 'category', 'polarity']]

df_train = pd.DataFrame({
    'polarity': df_train.groupby(['id', 'text', 'category'])['polarity'].apply(list),
}).reset_index()

df_train.head()

# Prepare Labels

In [ ]:
mlb = MultiLabelBinarizer()

y_train = mlb.fit_transform(df_train.polarity)
print(y_train)
y_train.shape

# Extract Features

In [ ]:
def lemmatize(x):
    return ' '.join([token.lemma_ for token in nlp(x)])

def select_column(df, column):
    return df.loc[:, column].values


def reshape_array(array, shape):
    return array.reshape(*shape)

In [ ]:
pipeline = make_union(
    make_pipeline(
        FunctionTransformer(select_column, kw_args={'column': 'text'}),
        TfidfVectorizer(preprocessor=lemmatize, stop_words='english', ngram_range=(1, 3)),
    ),
    make_pipeline(
        FunctionTransformer(select_column, kw_args={'column': 'category'}),
        FunctionTransformer(reshape_array, kw_args={'shape': (-1, 1)}),
        OneHotEncoder(),
    ),
)

x_train = pipeline.fit_transform(df_train).toarray()

x_train.shape

# Cross Validatation

In [ ]:
scoring = ['precision_micro', 'recall_micro', 'f1_micro']

clf = OneVsRestClassifier(LogisticRegression())

scores = cross_validate(clf, x_train, y_train, scoring=scoring, cv=3)

print('Micro Precision Score\t', np.average(scores['test_precision_micro']))
print('Micro Recall Score\t', np.average(scores['test_recall_micro']))
print('Micro F1 Score\t\t', np.average(scores['test_f1_micro']))

# -----------------------------------------

# Without #AspectCategory As Feature

# Micro Precision Score	 0.569311128799821
# Micro Recall Score	 0.5921557834734619
# Micro F1 Score		 0.5804751685040593

# -----------------------------------------

# With #AspectCategory as Feature

# Micro Precision Score	 0.5703663218941576
# Micro Recall Score	 0.5917189114769569
# Micro F1 Score		 0.5808142238635255

# Train Model

In [ ]:
clf.fit(x_train, y_train)

# Evaluate Model

In [ ]:
test_ds_path = DATA_PATHS['asba.semeval16.raw.test.gold']

df_test = load_dataset(test_ds_path)

df_test = df_test.loc[:, ['id', 'text', 'category', 'polarity']]

df_test = pd.DataFrame({
    'polarity': df_test.groupby(['id', 'text', 'category'])['polarity'].apply(list),
}).reset_index()

df_test.head()

In [ ]:
# Testing
x_test = pipeline.transform(df_test).toarray()

y_pred = clf.predict(x_test)

y_pred.shape

In [ ]:
y_true = mlb.transform(df_test.polarity)

y_true.shape

In [ ]:
('f1_score', f1_score(y_true, y_pred, average='micro'))